In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import sys
sys.path.append('/jupyter/ext')
import stackql
%load_ext stackql

In [ ]:
sprint_number = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Sprint Number',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    disabled=False
)

display(sprint_number)
display(start_date)
display(end_date)